In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfE = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')
dfT = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

/home/soledad-escobar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
#OHE a columna event
dfD = dfE[['person', 'event']]
dfD = pd.get_dummies(dfD, columns=['event']).groupby('person').sum()

In [85]:
#le agrego las features de event a las labels
dfTD = dfD.sort_values('person').merge(dfT,how = 'left', on='person')

In [86]:
#Agrego columna con ultima visita al sitio
temp = dfE.groupby('person')['timestamp'].max().to_frame()
dfTDb = dfTD.merge(temp, on='person')
dfTDb['month'] = dfTDb['timestamp'].str[5:7]
dfTDb['day'] = dfTDb['timestamp'].str[8:10]
dfTDb[['day','month']] = dfTDb[['day','month']].apply(pd.to_numeric)
dfTDb['last_visit'] = dfTDb['month']*30 + dfTDb['day']
dfTDb = dfTDb.drop(['timestamp','month','day'], axis=1)

In [178]:
#Agrego columna con ultima compra
temp = dfE.loc[dfE['event'] == 'conversion'].groupby('person')['timestamp'].max().to_frame()
dfTDc = dfTDb.merge(temp, how='left', on='person')
dfTDc = dfTDc.fillna('000000000000000000')
dfTDc['month'] = dfTDc['timestamp'].str[5:7]
dfTDc['day'] = dfTDc['timestamp'].str[8:10]
dfTDc[['day','month']] = dfTDc[['day','month']].apply(pd.to_numeric)
dfTDc['last_conversion'] = dfTDc['month']*30 + dfTDc['day']
dfTDc = dfTDc.drop(['timestamp','month','day', 'label'], axis=1)

In [179]:
dfTDc.head()

,person,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site,last_visit,last_conversion
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,167,0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0,181,0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,116,0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0,176,176
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,179,0


# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

# Me quedo solo con los datos de las pñersonas que tienen labels
df_TDc = dfTDc.merge(dfT, on = 'person')

df_y = df_TDc['label']
df_X = df_TDc.drop(['person', 'label'], axis=1)


GBC = GradientBoostingClassifier(random_state=23,learning_rate=0.06,subsample=0.75, min_samples_split=8)
predicciones = GBC.predict(X_test)
roc_auc_score(y_test, predicciones)

# Agrego nuevos features

## Analizo las fechas y los horarios 

In [181]:
# Agrego columnas para separar los meses y la hora
datos = dfE.loc[:,['timestamp', 'event', 'person']]
datos['date_time'] = pd.to_datetime(datos['timestamp'], format='%Y%m%d %H:%M:%S.%f')
datos['mes'] = datos['date_time'].dt.month
datos['hora'] = datos['date_time'].dt.hour

In [182]:
#Agrego columnas para ver el momento del dia en el que se generaron las conversiones
features = datos.loc[ datos['event'] == 'conversion', ['person', 'hora']]
features['comp_mañana'] = features['hora'].apply(lambda x: 1 if ((x >= 6) & (x < 12)) else 0)
features['comp_tarde'] = features['hora'].apply(lambda x: 1 if ((x >= 12) & (x <20)) else 0)
features['comp_noche'] = features['hora'].apply(lambda x: 1 if ((x >= 20) & (x <= 23)) else 0)
features['comp_madrugada'] = features['hora'].apply(lambda x: 1 if ((x >= 0) & (x < 6)) else 0)
features = features.drop('hora', axis = 'columns').groupby('person').sum()

In [183]:
# Hago un join con los features anteriores
df_feat = pd.merge(dfTDc, features, on = 'person', how = 'left').fillna(0)
# Agrego la cantidad de eventos que generaron las personas
df_feat['total_events'] = (np.sum(df_feat.iloc[:,2:13], axis = 'columns'))
df_feat.head()

,person,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,event_viewed product,event_visited site,last_visit,last_conversion,comp_mañana,comp_tarde,comp_noche,comp_madrugada,total_events
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,167,0,0.0,0.0,0.0,0.0,173.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0,181,0,0.0,0.0,0.0,0.0,614.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,116,0,0.0,0.0,0.0,0.0,125.0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0,176,176,0.0,1.0,0.0,0.0,381.0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,179,0,0.0,0.0,0.0,0.0,195.0


# XGBoost Classifier

In [184]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Me quedo solo con los datos de las pñersonas que tienen labels
df_features = df_feat.merge(dfT, on = 'person')

y = df_features['label']
X = df_features.drop(['person', 'label'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2)

In [185]:
XGB1 = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.05,
                max_depth = 5, gamma = 4, n_estimators = 100).fit(X_train,y_train)

Realizo las predicciones y luego calculo el score

In [187]:
from sklearn.metrics import roc_auc_score
preds = XGB1.predict_proba(X_test)[:,1]
roc_auc_score(y_test, preds)

0.8473519770395687

In [188]:
scores = cross_val_score(XGB1 , X, y, scoring = "roc_auc", cv = 5)
scores.mean()

0.8539767089769418

## Agrego más features sobre marcas y modelos de los productos

Como se tienen muchos modelos diferentes pero corresponden a una cantidad acotada de marcas, trabajo sobre las marcas para crear nuevos features. Creo una columna por cada marca existente y para cada persona coloco el promedio de compra para esa marca.

In [192]:
#Creo columnas para las marcas existentes
compras_por_marcas = dfE.loc[datos['event'] == 'conversion', ['model', 'person']]
compras_por_marcas['Samsung_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('samsung' in x.lower()) else 0)
compras_por_marcas['iPhone_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('iphone' in x.lower()) else 0)
compras_por_marcas['Motorola_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('moto' in x.lower()) else 0)
compras_por_marcas['Lenovo_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('lenovo' in x.lower()) else 0)
compras_por_marcas['LG_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('lg' in x.lower()) else 0)
compras_por_marcas['Sony_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('sony' in x.lower()) else 0)
compras_por_marcas['Asus_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('asus' in x.lower()) else 0)
compras_por_marcas['iPad_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('iPad' in x.lower()) else 0)
compras_por_marcas['Quantum_mean'] = compras_por_marcas['model'].apply(lambda x: 1 if ('quantum' in x.lower()) else 0)
features = compras_por_marcas.groupby('person').mean()

In [194]:
# Hago join con los features anteriores 
new_features = pd.merge(df_feat, features, on = 'person', how = 'left').fillna(0)
new_features.columns

Index(['person', 'event_ad campaign hit', 'event_brand listing',
       'event_checkout', 'event_conversion', 'event_generic listing',
       'event_lead', 'event_search engine hit', 'event_searched products',
       'event_staticpage', 'event_viewed product', 'event_visited site',
       'last_visit', 'last_conversion', 'comp_mañana', 'comp_tarde',
       'comp_noche', 'comp_madrugada', 'total_events', 'Samsung_mean',
       'iPhone_mean', 'Motorola_mean', 'Lenovo_mean', 'LG_mean', 'Sony_mean',
       'Asus_mean', 'iPad_mean', 'Quantum_mean'],
      dtype='object')

## Vuelvo a correr XGBoost para ver si con los nuevos features mejoran las predicciones

In [296]:
# Me quedo solo con los datos de las personas que tienen labels
df_new_features = new_features.merge(dfT, on = 'person')

y = df_new_features['label']
X = df_new_features.drop(['person', 'label'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2)

In [307]:
XGB2 = GradientBoostingClassifier(learning_rate = 0.1,
                max_depth = 5, n_estimators = 300).fit(X_train,y_train)

Calculo las predicciones y los score

In [308]:
predicciones = XGB2.predict(X_test)
roc_auc_score(y_test, predicciones)

0.5200436362514971

In [ ]:
scores = cross_val_score(XGB2 , X, y, scoring = "roc_auc", cv = 10)
scores.mean()

In [242]:
person_test = pd.read_csv('/home/soledad-escobar/Descargas/Orga_datos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

In [284]:
set_person_test = new_features.merge(person_test, on = 'person')
set_test = set_person_test.drop(['person'], axis = 'columns')

In [285]:
set_test.shape

(19415, 27)

## Vuelvo a entrenar el algoritmo

In [281]:
# Me quedo solo con los datos de las personas que tienen labels
df_new_set = new_features.merge(dfT, on = 'person')

df_y = df_new_set['label']
df_X = df_new_set.drop(['person', 'label'], axis=1)

X_train, X_test, y_train, y_test = \
    train_test_split(df_X, df_y, test_size = 0.2)

In [282]:
XGB2 = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, gamma = 5, n_estimators = 100).fit(X_train,y_train)

## Realizo las predicciones

In [247]:
prediccion_final = XGB2.predict_proba(set_test)[:,1]

In [286]:
personas = list(set_person_test['person'])

In [292]:
import csv
datos = []
for i in range(19415):
    datos.append((personas[i], prediccion_final[i]))

In [293]:
with open('predicciones.csv', 'w', newline='') as csvSalida:
    salida = csv.writer(csvSalida)
    salida.writerow(['person', 'label'])
    salida.writerows(datos)

In [294]:
len(personas)

19415

In [295]:
len(prediccion_final)

19415